In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install --quiet langchain_openai transformers sentence_transformers torch pipeline langchain_community
!pip install --quiet langchain unstructured[all-docs] pydantic lxml langchainhub fastapi
!pip install pillow==8.3.2
!pip install typer==0.3.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
  Using cached Pillow-8.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.15 requires pillow>=9.0.0, but you have pillow 8.3.2 which is incompatible.
pdfplumber 0.11.0 requires Pillow>=9.1, but you have pillow 8.3.2 which is incompatible.
pikepdf 8.15.1 requires Pillow>=10.0.1, but you have pillow 8.3.2 which is incompatible.
pillow-heif 0.16.0 requires pillow>=9

  Using cached typer-0.3.0-py3-none-any.whl (21 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.5 requires click>=8.0, but you have click 7.1.2 which is incompatible.
chromadb 0.5.0 requires typer>=0.9.0, but you have typer 0.3.0 which is incompatible.
dask 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
distributed 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.3.0 which is in

In [2]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
import openai
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import os


In [4]:
from unstructured.partition.pdf import partition_pdf
elements = partition_pdf("/content/manual.pdf",
                         extract_images_in_pdf=True,
                         infer_table_structure=True,
                         chunking_strategy="by_title",
                         max_characters=1000,
                         new_after_n_chars=1000,
                         strategy="hi_res"
                         )

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 36,
 "<class 'unstructured.documents.elements.Table'>": 6,
 "<class 'unstructured.documents.elements.TableChunk'>": 26}

In [6]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
for element in elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))
print(table_elements[0].text)
# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

32
Maintenance Record Sheet
36


In [7]:
model_name="sentence-transformers/all-mpnet-base-v2"
embeddings=HuggingFaceEmbeddings(model_name=model_name)

In [8]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

768

In [9]:
!pip install langchain-chroma

In [10]:
!pip install chromadb
import chromadb
chroma_client = chromadb.Client()
#collection = chroma_client.create_collection(name="summaries")

In [11]:
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
LLM_Model="google/flan-t5-large"
tokenizer=AutoTokenizer.from_pretrained(LLM_Model)
model=AutoModelForSeq2SeqLM.from_pretrained(LLM_Model,torch_dtype=torch.float32)
pipe=pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=300,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
    )
llm=HuggingFacePipeline(pipeline=pipe)
llm

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7e0227f07340>)

In [12]:
llm("What is the capital of India")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'calcutta'

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt_text="""You are an assistant tasked with summarizing tables and text provided in the Car Manual. \
Give a concise summary of the table or text.Table or Text chunk: {element}"""
prompt=ChatPromptTemplate.from_template(prompt_text)
summarize_chain={"element": lambda x: x} | prompt | llm | StrOutputParser(max_length=500)

In [14]:
!pip install langchain_core
from langchain_core.prompts import ChatPromptTemplate
prompt2=ChatPromptTemplate.from_template("""
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.\
If the answer is from the tables, Display it in table format\
Provide answers in the human readable format.
<context>
{context}
</context>
Question: {input}
""")

In [15]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=embeddings
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)
retriever

MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e02201231c0>, docstore=<langchain_core.stores.InMemoryBaseStore object at 0x7e0220122da0>)

In [16]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
tables = [i.text for i in table_elements]
print(tables)
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
table_summaries

['Maintenance Record Sheet', '“uBIS “yoo) “uBIS “APY 48S dajeap Buildin -J8S JO OWEN auop sedoy jo sile}9q Arobaye9 djedoy swy “ON OU', '‘ubls 4aje2ep BUIDIA | guop seday | Alobaye9 “uBis “YoOL ‘APY JES |-18S Jo aWeN| Jo sjlejeq ayeday swy ‘ONON', 'Exterior overview (i) ........................................................................................ 2-2 Exterior overview (ii) ....................................................................................... 2-3 Interior overview ............................................................................................. 2-6 Instrument panel overview ............................................................................ 2-8 Instrument panel overview ............................................................................ 2-9 Engine compartment .....................................................................................2-10 Dimensions .......................................................................

['The Maintenance Record Sheet is a record of the maintenance of the vehicle.',
 '“uBIS “APY 48S dajeap Buildin -J8S JO OWEN auop sedoy jo sile9q Arobaye9 djedoy swy “ON OU',
 'The following are the names of the four cylinders of the car:',
 'Exterior overview (i) 2-2 Exterior overview (ii) 2-3 Interior overview ..................................................................................... 2-6 Instrument panel overview ............................................................................. 2-8 Instrument panel overview .............................................................................. 2-9 Engine compartment .....................................................................................2-10 Dimensions ......................................................................................................2-11 Engine specification ......................................................................................2-11 Bulb wattage',
 'The following informati

In [17]:
# Apply to texts
texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

In [18]:
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [Document(page_content=s,metadata={id_key: table_ids[i]}) for i, s in enumerate(table_summaries)]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt2
    | llm
    | StrOutputParser()
)
chain

{
  context: MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e02201231c0>, docstore=<langchain_core.stores.InMemoryBaseStore object at 0x7e0220122da0>),
  input: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.If the answer is from the tables, Display it in table formatProvide answers in the human readab

In [20]:
chain.invoke("What is Vehicle Identification Number(VIN)?")

Token indices sequence length is longer than the specified maximum sequence length for this model (702 > 512). Running this sequence through the model will result in indexing errors


"The vehicle identification number (VIN) is the number used in registering your car and in all legal matters pertaining to its ownership, etc. The number is punched on the floor under the front right seat. To check the number, pull out the slit part of carpet.nn2-16nnVEH TLECERTFLATDN LABEL (IF EQUIPPED)nnS BS 224nnThe vehicle certification label attached on the driver’s (or front passenger’s side center pillar gives the vehicle identification number (VIN).nnTIRE SPECIFICATION AND PRESSURE LABELnnThe tires supplied on your new vehicle are chosen to provide the best performance for normal driving. The tire label located on the driver’s side center pillar gives the tire pressures recommended for your car.nn02nn2-17nVehicle Information', '....................................................................................... 2-12 Tires and wheels ...................................................................................... 2-13 Recommended lubricants and capacities ..............

In [21]:
chain.invoke("What are the SAFETY AND VEHICLE DAMAGE WARNING")

"This manual includes information titled as DANGER, WARNING, CAUTION and NOTICE. These titles indicate the following:nnDANGERnnDANGER indicates a hazardous situation which, if not avoided, will result in death or serious injury.nnAwnnWARNINGnnWARNING indicates a hazardous situation which, if not avoided, could result in death or serious injury.nncautonnnCAUTIONnnCAUTION indicates a hazardous situation which, if not avoided, could result in minor or moderate injury.nnNOTICE NOTICE indicates a situation which, if not avoided, could result in vehicle damage.nnTable of ContentsnnHyundai Warranty PolicynnVehicle Information Safety System Instrument Cluster Convneince Features Driving Your Vehicle Driver Assistance System Emergency Situations Maintenance 2 3 4 5 6 7 8 9 nn1', '2. Petrol or gasohol containing methanol. 3. Leaded fuel or leaded gasohol.nnCAUTIONnnNever use gasohol which contains methanol. Discontinue use of any gasohol product which impairs d"

In [ ]:
chain.invoke("What are the switches are used to control")

In [ ]:
chain.invoke("What are the features of the vehicle")

In [ ]:
chain.invoke("What are the Specifications of Tires and Wheels")

In [ ]:
chain.invoke("Which lubricants and fluids are recommended for use in your vehicle")

In [ ]:
response=chain.invoke("What are the Specifications of Tires and Wheels")

In [ ]:
from IPython.display import Markdown, display
display(Markdown(f"{response}"))

In [ ]:
from IPython.display import Markdown, display, display_html, HTML, display_markdown
display(Markdown(response))

In [ ]:
from IPython.display import Markdown, display
display(Markdown(f"{response[1]}"))

# **Chatbot**

In [22]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [ ]:
retriever

In [23]:
def load_db(file,chain_type,k):
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    model_name="sentence-transformers/all-mpnet-base-v2"
    embeddings=HuggingFaceEmbeddings(model_name=model_name)
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    LLM_Model="google/flan-t5-large"
    tokenizer=AutoTokenizer.from_pretrained(LLM_Model)
    model=AutoModelForSeq2SeqLM.from_pretrained(LLM_Model,torch_dtype=torch.float32)
    pipe=pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=300,
        temperature=0,
        top_p=0.95,
        repetition_penalty=1.15
    )
    llm=HuggingFacePipeline(pipeline=pipe)

    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa


In [24]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "/content/manual.pdf"
        self.chain = load_db(self.loaded_file,"stuff",4)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.chain({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=1000)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=1000, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return


In [25]:
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

In [26]:
!pip install -U docarray

In [27]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=700),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithME')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=700, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-24-552a5b105dc4>:38: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=1000, style={'background-color': '#F6F6F6'}))
<ipython-input-24-552a5b105dc4>:38: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=1000, style={'background-color': '#F6F6F6'}))
<ipython-input-24-552a5b105dc4>:38: PanelDeprecationWarning: 'style' is deprecated and wil